In [1]:
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis import align
import Bio.SeqIO

### Input pdb files:

In [2]:
pdb1 = 'input_files/5weo_final_homology_model.pdb'
pdb2 = 'input_files/7eo_final_homology_model.pdb'

### Selections for aligment:
(syntax: https://docs.mdanalysis.org/stable/documentation_pages/selections.html )

In [27]:
sel_pdb1 = 'segid A and resid 73:310'
sel_pdb2 = 'segid A and resid 30:293'

### Indexes of first residues in the selections:

In [28]:
resid_of_first_residue_in_sel_pdb1 = 73
resid_of_first_residue_in_sel_pdb2 = 30

### Output fasta file for uniprot:

In [29]:
fasta_file = 'fasta_for_uniprot.fasta'

### Run next cell to create fasta_file and to print its content

In [30]:
u1 = mda.Universe(pdb1)
u2 = mda.Universe(pdb2)

subset1 = u1.select_atoms(sel_pdb1)
subset2 = u2.select_atoms(sel_pdb2)


record1 = subset1.residues.sequence(id='pdb1')
record2 = subset2.residues.sequence(id='pdb2')

Bio.SeqIO.write([record1, record2], fasta_file, 'fasta')

with open(fasta_file,'r') as file:
    print(file.read())

>pdb1 <unknown description>
NMLTSFCGALHVCFITPSFPVDTSNQFVLQLRPELQDALISIIDHYKWQKFVYIYDADRG
LSVLQKVLDTAAEKNWQVTAVNILTTTEEGYRMLFQDLEKKKERLVVVDCESERLNAILG
QIIKLEKNGIGYHYILANLGFMDIDLNKFKESGANVTGFQLVNYTDTIPAKIMQQWKNSD
ARDHTRVDWKRPKYTSALTYDGVKVMAEAFQSLRRQRIDISRRGNAGDCLANPAVPWG
>pdb2 <unknown description>
LPLDVNVVALLMNRTDPKSLITHVCDLMSGARIHGLVFGDDTDQEAVAQMLDFISSHTFV
PILGIHGGASMIMADKDPTSTFFQFGASIQQQATVMLKIMQDYDWHVFSLVTTIFPGYRE
FISFVKTTVDNSFVGWDMQNVITLDTSFEDAKTQVQLKKIHSSVILLYCSKDEAVLILSE
ARSLGLTGYDFFWIVPSLVSGNTELIPKEFPSGLISVSYDDWDYSLEARVRDGIGILTTA
ASSMLEKFSYIPEAKASCYGQMER



### Create aligment with uniprot
Copy output from previous cell to https://www.uniprot.org/align and then press "Run Align". 

When the aligment is ready, download it as an uncompressed text file. Set "aligment_file" to equal the file you just dowloaded.

In [31]:
aligment_file = 'aligment.aln'

### Run next cell to get common AtomGroups

In [41]:
def read_seqs_from_aligment_file(aligment_file, id1='pdb1', id2='pdb2'):
    seq1, seq2 = '', ''
    with open(aligment_file,'r') as file:
        for line in file:
            s = line.split()
            if len(s) > 1:
                if s[0] == id1:
                    seq1 += s[1]
                elif s[0] == id2:
                    seq2 += s[1]
    return seq1, seq2


def find_common_atom_select(seq1,seq2, i1=0, i2=0):

    backbone1 = ''
    whole_res1 = ''
    
    backbone2 = ''
    whole_res2 = ''
    for aa1,aa2 in zip(seq1,seq2):

        bool1 = aa1 != '-'
        bool2 = aa2 != '-'

        if bool1 and bool2:
            i1 += 1
            i2 += 1
            if aa1 == aa2:
                whole_res1 += ' {}'.format(i1)
                whole_res2 += ' {}'.format(i2)
            else:
                backbone1 += ' {}'.format(i1)
                backbone2 += ' {}'.format(i2)
                

        elif bool2: i2 += 1
        elif bool1: i1 += 1
    
    
    if backbone1 == '':
        if whole_res1 == '':
            sel1 = ''
        else:
            sel1 = 'resid {}'.format(whole_res1)
            
    elif whole_res1 == '':
        sel1 = 'backbone and resid {}'.format(backbone1)
    else:
        sel1 = '(backbone and resid {}) or (resid {})'.format(backbone1,whole_res1)
    
    
    if backbone2 == '':
        if whole_res2 == '':
            sel2 = ''
        else:
            sel2 = 'resid {}'.format(whole_res2)
            
    elif whole_res2 == '':
        sel2 = 'backbone and resid {}'.format(backbone2)
    else:
        sel2 = '(backbone and resid {}) or (resid {})'.format(backbone2,whole_res2)
    
    
    return sel1, sel2

seq1, seq2 = read_seqs_from_aligment_file(aligment_file, id1='pdb1', id2='pdb2')

i1 = resid_of_first_residue_in_sel_pdb1 - 1
i2 = resid_of_first_residue_in_sel_pdb2 - 1
sel1, sel2 = find_common_atom_select(seq1,seq2, i1=i1, i2=i2)

common_sel1 = subset1.select_atoms(sel1)
common_sel2 = subset2.select_atoms(sel2)

common_sel1, common_sel2

(<AtomGroup with 1238 atoms>, <AtomGroup with 1237 atoms>)

### Write AtomGroups to Gromacs ndx files

In [42]:
ndx_file1 = 'hih.ndx'
ndx_file2 = 'hah.ndx'

name1 = 'transmembrane_part'
name2 = 'transmembrane_part'

In [43]:
with mda.selections.gromacs.SelectionWriter(ndx_file1, mode='w') as ndx1:
    ndx1.write(common_sel1, name=name1)
    
with mda.selections.gromacs.SelectionWriter(ndx_file2, mode='w') as ndx2:
    ndx2.write(common_sel2, name=name1)

### Write AtomGroups to pdb files

In [44]:
pdb_common1 = 'common1.pdb'
pdb_common2 = 'common2.pdb'

common_sel1.write(pdb_common1)
common_sel2.write(pdb_common2)